## Step 1: Setup env, SQL connection and analyze SQL database table data

Import all necessary libraries and establish connection to the SQL database (score.db)

In [ ]:
# Import necessary libraries
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Set path to SQL database
db_path = "../data/score.db"

# Create connection to SQL database
conn = sqlite3.connect(db_path)

# Set pandas options for better readability
pd.set_option('display.max_columns', None) # Display all columns in DataFrames
pd.set_option('display.max_rows', 100)     # Limit number of rows displayed

# Setup matplotlib and seaborn for inline visualization
%matplotlib inline
sns.set(style="whitegrid")

Explore structure of database by listing all available tables before any further actions

In [ ]:
# Query to list all tables in the database
query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql(query, conn)

# Display list of tables
tables

Since there is only 'score' table in the database, the first few rows can be previewed to understand its structure

In [ ]:
# Preview first few rows of 'score' table
query = "SELECT * FROM score LIMIT 10;"
df_score = pd.read_sql(query, conn)

# Display first 10 rows of the table
df_score.head()

The schema of the selected table is retrieved to understand the columns and their data types

In [ ]:
# Get schema of 'score' table
query = "PRAGMA table_info(score);"
schema = pd.read_sql(query, conn)

# Display schema information
schema

## Step 2: Perform Exploratory Data Analysis (EDA) on 'score' table

Firstly, check the number of rows in 'score' table to get a sense of the size of the data

In [ ]:
# Get row_count of 'score' table
row_count_query = "SELECT COUNT(*) AS count FROM score;"
row_count = pd.read_sql_query(row_count_query, conn)

# Display number of rows in 'score' table
row_count

Next, load all the data into a DataFrame for actual data analysis

In [ ]:
# Get all data from 'score' table
score_data_query = "SELECT * FROM score;"
score_data = pd.read_sql_query(score_data_query, conn)

There are some columns in 'score' table that are irrelevant in predicting the students' results <br>
These columns will be dropped from the DataFrame:
- index
- number_of_siblings
- student_id
- gender
- n_male
- n_female
- age
- bag_color

In [ ]:
irrelevant_col_list = ["index", "number_of_siblings", "student_id", "gender", "n_male", "n_female", "age", "bag_color"]

drop_col_score_data = score_data.drop(columns = irrelevant_col_list)

Then clean-up the data of missing cell info <br>
Example: If a cell in a row has missing value, then the row should be dropped

In [ ]:
# Check the number of rows that have missing data
missing_data = drop_col_score_data.isnull().sum()

# Display number of affected rows
print("Missing values in each column: ")
print(missing_data[missing_data > 0])

In [ ]:
# Drop rows if there are any missing values
cleaned_score_data = drop_col_score_data.dropna()

# Verify if all missing values are dropped
print("Missing values after dropping rows: ")
print(cleaned_score_data.isnull().sum())

The 'score' DataFrame is now free from empty cells <br>
However, there are non-numeric values in the table which cannot be used for correlation <br>
So they should be converted into a numeric representation

In [ ]:
# Create empty dict to add in converted value labels for plotting use
convert_label_dict = {}

Starting with direct_admission column:
- 0 - No
- 1 - Yes

Take into account different format of the same pattern (uppercases/lowercases)

In [ ]:
# Convert 'No' -> 0 and 'Yes' -> 1 in direct_admission column
cleaned_score_data["direct_admission"] = cleaned_score_data["direct_admission"].str.lower().replace({"no": 0, "yes": 1})

# Check if anything is missed from direct_admission column by checking for unique values
cleaned_score_data["direct_admission"].unique()

# Add conversion to dict
convert_label_dict["direct_admission"] = ['0 - No', '1 - Yes']

Next is CCA column:
- 0 - Sports
- 1 - Arts
- 2 - Clubs
- 3 - None

Take into account different format of the same pattern (uppercases/lowercases)

In [ ]:
# Convert 'Sports' -> 0, 'Arts' -> 1, "Clubs" -> 2 and "None" -> 3 in CCA column
cleaned_score_data["CCA"] = cleaned_score_data["CCA"].str.lower().replace({"sports": 0, "arts": 1, "clubs": 2, "none": 3})

# Check if anything is missed from CCA column by checking for unique values
cleaned_score_data["CCA"].unique()

# Add conversion to dict
convert_label_dict["CCA"] = ['0 - Sports', '1 - Arts', '2 - Clubs', '3 - None']

Replace learning_style column with following format:
- 0 - Visual
- 1 - Auditory

In [ ]:
# Convert 'Visual' -> 0 and 'Auditory' -> 1 in learning_style column
cleaned_score_data["learning_style"] = cleaned_score_data["learning_style"].str.lower().replace({"visual": 0, "auditory": 1})

# Check if anything is missed from learning_style column by checking for unique values
cleaned_score_data["learning_style"].unique()

# Add conversion to dict
convert_label_dict["learning_style"] = ['0 - Visual', '1 - Auditory']

Replace tuition column with following format:
- 0 - N/No
- 1 - Y/Yes

In [ ]:
# Convert 'No'/'N' -> 0 and 'Yes'/'Y' -> 1 in tuition column
cleaned_score_data["tuition"] = cleaned_score_data["tuition"].str.lower().replace({"no": 0, "yes": 1, "n": 0, "y": 1})

# Check if anything is missed from tuition column by checking for unique values
cleaned_score_data["tuition"].unique()

# Add conversion to dict
convert_label_dict["tuition"] = ['0 - No/N', '1 - Yes/Y']

Replace mode_of_transport column with following format:
- 0 - Walk
- 1 - Public Transport
- 2 - Private Transport

In [ ]:
# Convert 'Walk' -> 0, 'Public Transport' -> 1 and "Private Transport" -> 2 in mode_of_transport column
cleaned_score_data["mode_of_transport"] = cleaned_score_data["mode_of_transport"].str.lower().replace({"walk": 0, "public transport": 1, "private transport": 2})

# Check if anything is missed from mode_of_transport column by checking for unique values
cleaned_score_data["mode_of_transport"].unique()

# Add conversion to dict
convert_label_dict["mode_of_transport"] = ['0 - Walk', '1 - Public Transport', '2 - Private Transport']

Since sleep_time & wake_time columns are the only ones that are of non int/float values, they will need to be converted to numeric value <br>
These two columns will be used to calculate the number of hours of sleep each student has to try and get a correlation result

In [ ]:
# Create function to calculate sleep hours
def cal_sleep_hours(sleep_time, wake_time):
    # Convert sleep_time and wake_time to datetime objects
    sleep_time = datetime.strptime(sleep_time, '%H:%M')
    wake_time = datetime.strptime(wake_time, "%H:%M")

    # If wake_time is earlier than sleep_time, assume wake_time is on the next day
    if wake_time < sleep_time:
        wake_time += pd.Timedelta(days = 1)

    # Calculate difference in hours
    sleep_duration = (wake_time - sleep_time).total_seconds() / 3600 # Convert seconds to hours

    return sleep_duration

# Apply function to calculate sleep hours for each row
cleaned_score_data["sleep_hours"] = cleaned_score_data.apply(lambda row: cal_sleep_hours(row["sleep_time"], row["wake_time"]), axis = 1)

# Drop sleep_time and wake_time columns since they cannot be correlated
time_col_list = ["sleep_time", "wake_time"]
cleaned_score_data = cleaned_score_data.drop(columns = time_col_list)

# Check updated DataFrame
cleaned_score_data

It can now be used to generate summary statistics to check on the mean, median, 25%, etc

In [ ]:
# Get summary statistics from 'score' DataFrame
summary_stats = cleaned_score_data.describe()

# Display summary statistics
summary_stats

## Step 3: Analyze the patterns and distributions in 'score' DataFrame

Plot histograms for each numeric column against final_test to understand their distributions

In [ ]:
# Define the result column name
result_col = "final_test"

# Select numeric columns only (excluding the result column)
numeric_cols = cleaned_score_data.select_dtypes(include=['float', 'int']).columns
numeric_cols = [col for col in numeric_cols if col != result_col]

for col in numeric_cols:
    sns.scatterplot(data = cleaned_score_data, x = col, y = result_col, hue = col)
    plt.title(f"Distribution of {col} against result")
    plt.xlabel(col)
    plt.ylabel(result_col)
    if col in convert_label_dict:
        plt.legend(labels = convert_label_dict[col], title = "Legend:", loc = "best")
    else:
        plt.legend(title = "Legend:", loc = "best")
    plt.show()

As direct_admission, CCA, learning_style, tuition and mode_of_transport columns cannot be used to draw any meaningful correlation results, they will be dropped

In [ ]:
# Populate list with columns to be dropped
exclude_col_list = ["direct_admission", "CCA", "learning_style", "tuition", "mode_of_transport"]

# Drop columns
fil_score_data = cleaned_score_data.drop(columns = exclude_col_list)

# Check DataFrame
fil_score_data

Check scatterplot of remaining data to determine correlation again

In [ ]:
# Select numeric columns only (excluding the result column)
new_numeric_cols = fil_score_data.select_dtypes(include=['float', 'int']).columns
new_numeric_cols = [col for col in new_numeric_cols if col != result_col]

for col in new_numeric_cols:
    sns.scatterplot(data = fil_score_data, x = col, y = result_col, hue = col)
    plt.title(f"Distribution of {col} against result")
    plt.xlabel(col)
    plt.ylabel(result_col)
    if col in convert_label_dict:
        plt.legend(labels = convert_label_dict[col], title = "Legend:", loc = "best")
    else:
        plt.legend(title = "Legend:", loc = "best")
    plt.show()

Perform matrix correlation between different columns

In [ ]:
fil_corr_matrix = fil_score_data.corr()
sns.heatmap(fil_corr_matrix, annot = True, cmap = "coolwarm")
plt.title("Correlation Matrix")
plt.show()